# 참조자료

-  https://smartshk.tistory.com/19


-  https://www.baeldung.com/kotlin/threads-coroutines

## 코루틴 모듈 올리기

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.1")

## 모듈 설치 확인하기

In [2]:
:classpath

Current classpath (14 paths):
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/lib-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/api-0.11.0-61.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.7.0-dev-1825.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.6.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/dahlmoon/anaconda3/envs/kotlin/lib/python3.8/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.6.0.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx/kotlinx-coroutines-core/1.6.1/kotlinx-coroutines-core-1.6.1.jar
/Users/dahlmoon/.m2/repository/org/jetbrains/kotlinx

## 일시중단함수 작성 

In [3]:
import kotlinx.coroutines.*
import kotlin.system.measureTimeMillis 

suspend fun doSomethingOne(): Int {               // 일시중단함수    
    delay(100L)                                   // 일시정지
    println("일시중단함수 실행 1")
    return 13
}

suspend fun doSomethingTwo(): Int {               // 일시중단함수 
    delay(300L)                                   // 일시정지 
    println("일시중단함수 실행 2")
    return 29
}

fun main() {
    GlobalScope.launch(Dispatchers.Default) {     // 전역스코프에 코루틴 빌더 
            val time = measureTimeMillis {        // 처리시간 계산 
                val one = doSomethingOne()        // 함수 호출 
                val two = doSomethingTwo()
                println( "실행결과 1 : $one")       // 실행결과 출력 
                println( "실행결과 2 : $two")
            }
            delay(1000)                           // 일시 지연 
            println("총 실행시간 :  $time ms.")      // 함수실행결과 출력 
    }
}

println("### 작업 종료  ###")
main()
Thread.sleep(2000)                                 // 전체 처리 결과 대기

### 작업 종료  ###
일시중단함수 실행 1
일시중단함수 실행 2
실행결과 1 : 13
실행결과 2 : 29
총 실행시간 :  413 ms.


## 일시중단메소드 작성

In [4]:
import kotlinx.coroutines.*

class Suspend {                               // 클래스 정의 
    fun compute() {                           // 일반 메소드
        println("일반함수 호출 ")
    }
    suspend fun doSomethingOne(): Int {       // 일시중단 메소드
        delay(100L)                           // 일시중단함수 처리
        println("일시중단메소드 호출 1")           // 출력
        return 13                             // 반환값
    }

    suspend fun doSomethingTwo(): Int {       // 일시중단 메소드
        delay(300L)                           // 일시중단함수 처리 
        println("일시중단메소드 호출 2")           // 출력 
        return 29                             // 반환값
    }
}

fun main() { 
    val time = runBlocking {                   // 코루틴 구성 
        val sus = Suspend()                    // 객체 생성                 
        sus.compute()                          // 일반메소드 실행 
        val one = sus.doSomethingOne()         // 일시중단메소드 실행
        val two = sus.doSomethingTwo()
        delay(200)                             // 일시중단 처리
        println("메소드 처리 결과 : ${one + two}") // 일시중단메소드 결과 출력
    }
}
main()

일반함수 호출 
일시중단메소드 호출 1
일시중단메소드 호출 2
메소드 처리 결과 : 42


## 일시중단함수에 코루틴스코프 지정

In [5]:
import kotlinx.coroutines.*

fun main() = runBlocking {                           // 런블럭으로 코루틴 지정
    doWorld()                                        // 일시중단함수 호출   
}

suspend fun doWorld() =                              // 일시중단함수
    coroutineScope {                                 // 코루틴스코프 지정
        launch(Dispatchers.Default) {                // 코루틴 빌더에 코루틴 컨텍스트 지정
            delay(1000L)                             // 일시정지
            println("좋은 세상! : " 
                    + Thread.currentThread().name)   // 현재 코루틴 처리 
        }
        
        println("코틀린은 : " 
                    + Thread.currentThread().name)   // 현재 스레드 처리 
        
    }

main()

코틀린은 : Thread-20
좋은 세상! : DefaultDispatcher-worker-1


In [6]:
import kotlinx.coroutines.*

fun main34() = runBlocking {                      // 코루틴을 현재 스레드로 처리 
    withContext(Dispatchers.Default) {            // 코루틴 컨텍스트 변경 
        doWorld()                                 // 일시중단함수 호출 
    }
    println("Done")                               // 종료
    
    delay(3000)                                   // 모든 코루틴 종료 대기 
}

suspend fun doWorld() = coroutineScope {          // 코루틴 스코프 지정 
    launch {                                      // 코푸틴 빌더 
        delay(2000L)
        println("World 2 : "
                + Thread.currentThread().name)
    }
    launch {                                      // 코푸틴 빌더 
        delay(1000L)
        println("World 1 : "
                + Thread.currentThread().name)
    }
    println("Hello : " 
                + Thread.currentThread().name)
}

main34()

Hello : DefaultDispatcher-worker-1
World 1 : DefaultDispatcher-worker-1
World 2 : DefaultDispatcher-worker-1
Done
